# Guía de Ejercicios

In [1]:
from finrisk import QC_Financial_3 as Qcf
from IPython.display import Image
from IPython.core.display import HTML

## Equivalencia de Tasas de Interés

Utilizando la librería `QC_Financial_3` o directamente en Python, replicar los valores de la siguiente tabla:

In [2]:
Image(url="img/tabla_ejercicio.png", width=550, height=440)

## Construcción de Flujos

In [3]:
import pandas as pd

### Flujos Pata Fija

Replique en Python, sin usar la librería `QC_Financial_3` la siguiente tabla de desarrollo de la pata fija de un swap.

In [4]:
tabla = pd.read_excel('data/flujos_pata_fija.xlsx')

In [5]:
tabla.style.format({
    'nominal': '{0:,.0f}',
    'amortizacion': '{0:,.0f}',
    'interes': '{0:,.0f}',
    'flujo': '{0:,.0f}',
    'valor_tasa': '{0:,.4%}',
})

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2020-09-17,2021-03-17,2021-03-17,"10,000,000",0,"10,905",True,"10,905",USD,0.2181%,Lin30360
1,2021-03-17,2021-09-17,2021-09-17,"10,000,000",0,"10,905",True,"10,905",USD,0.2181%,Lin30360
2,2021-09-17,2022-03-17,2022-03-17,"10,000,000",0,"10,905",True,"10,905",USD,0.2181%,Lin30360
3,2022-03-17,2022-09-19,2022-09-19,"10,000,000","10,000,000","11,026",True,"10,011,026",USD,0.2181%,Lin30360


#### Solución

Lo importante es poder replicar las fechas y los intereses.

Para el campo `fecha_final` necesitamos una librería en Pytgon que nos permita sumar un número de meses a una fecha dada. Para encontrarla googleamos "add a certain amount of months to date in python". El primer link del resultado es: https://thispointer.com/add-months-to-a-date-in-python/ . Ahí se explica el uso del módulo `dateutil` y en particular de la clase `relativedelta`.

In [12]:
from datetime import date
from dateutil.relativedelta import relativedelta

In [16]:
f1 = date(2021, 2, 15)
f2 = f1 + relativedelta(months=1)
print(f2)

2021-03-15


Necesitamos, además, una función que imite el comoportamiento de `FOLLOWING`. Escribimos una función muy sencilla que a un sábado o un domingo asocia el lunes siguiente y a los días entre lunes y viernes inclusive los deja sin cambio.

In [17]:
def buss_day(fecha: date) -> date:
    """
    Imita el comportamiento de `FOLLOWING`.
    
    A un sábado o un domingo asocia el lunes siguiente y a los días entre lunes y viernes,
    inclusive, los deja sin cambio
    """
    if fecha.weekday() == 5:    # Sábado
        return fecha + relativedelta(days=2)
    elif fecha.weekday() == 6:  # Domingo
        return fecha + relativedelta(days=1)
    else:
        return fecha

In [19]:
# Domingo
f = date(1969, 1, 12)
print(buss_day(f))

# Sábado
f = date(1969, 1, 11)
print(buss_day(f))

1969-01-13
1969-01-13


Para el conteo de días en base 30360 googleamos "DAYS360 equivalent in python". El primer link del resultado es una pregunta en StackOverflow: https://stackoverflow.com/questions/51832672/pandas-excel-days360-equivalent . De ahí sacamos el siguiente código (el cual probaremos para este caso).

In [10]:
def days360(start_date, end_date, method_eu=False):
    """
    Calcula el número de días entre 2 fechas en base 30360.
    """
    start_day = start_date.day
    start_month = start_date.month
    start_year = start_date.year
    end_day = end_date.day
    end_month = end_date.month
    end_year = end_date.year

    if (
        start_day == 31 or
        (
            method_eu is False and
            start_month == 2 and (
                start_day == 29 or (
                    start_day == 28 and
                    start_date.is_leap_year is False
                )
            )
        )
    ):
        start_day = 30

    if end_day == 31:
        if method_eu is False and start_day != 30:
            end_day = 1

            if end_month == 12:
                end_year += 1
                end_month = 1
            else:
                end_month += 1
        else:
            end_day = 30

    return (
        end_day + end_month * 30 + end_year * 360 -
        start_day - start_month * 30 - start_year * 360)

In [22]:
f1 = date(2021, 2, 15)
f2 = date(2021, 3, 15)
f3 = date(2022, 2, 15)
print(days360(f1, f2))
print(days360(f1, f3))

30
360


In [28]:
date0 = date(2020, 9, 17)
start_date = date0
valor_tasa = .002181
result = []
for i in range(4):
    end_date = buss_day(date0 + relativedelta(months=(i + 1) * 6))
    nominal = 10000000
    amortizacion = 0.0 if i < 3 else nominal
    interes = nominal * valor_tasa * days360(start_date, end_date) / 360.0
    temp = (start_date, end_date, end_date, nominal, interes, amortizacion)
    result.append(temp)
    start_date = end_date

In [29]:
df_result = pd.DataFrame(
    result,
    columns=[
        'start_date',
        'end_date',
        'settlement_date',
        'nominal',
        'interes',
        'amortizacion'
    ])

In [30]:
df_result

,start_date,end_date,settlement_date,nominal,interes,amortizacion
0,2020-09-17,2021-03-17,2021-03-17,10000000,10905.000000,0.0
1,2021-03-17,2021-09-17,2021-09-17,10000000,10905.000000,0.0
2,2021-09-17,2022-03-17,2022-03-17,10000000,10905.000000,0.0
3,2022-03-17,2022-09-19,2022-09-19,10000000,11026.166667,10000000.0


### Flujos Pata OIS

Replique en Python, sin usar la librería `QC_Financial_3` la siguiente tabla de desarrollo de la pata cámara promedio (OIS de Chile) de un swap. Los valores del ICP se obtuvieron suponiendo una tasa interbancaria constante igual a 1.5%.

In [6]:
tabla = pd.read_excel('data/tabla_desarrollo_para_icp-ois.xlsx')

In [7]:
tabla.style.format({
    'nominal': '{0:,.0f}',
    'amortizacion': '{0:,.0f}',
    'interes': '{0:,.0f}',
    'flujo': '{0:,.0f}',
    'icp_inicial': '{0:,.2f}',
    'icp_final': '{0:,.2f}',
    'valor_tasa': '{0:,.4%}',
})

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2021-09-21,2022-03-21,2022-03-21,"1,000,000",0,True,"7,491",CLP,"10,000.00","10,075.00",1.4900%,"7,491",0,1,LinAct360
1,2022-03-21,2022-09-21,2022-09-21,"1,000,000",0,True,"7,513",CLP,"10,075.00","10,150.56",1.4700%,"7,513",0,1,LinAct360
2,2022-09-21,2023-03-21,2023-03-21,"1,000,000",0,True,"7,491",CLP,"10,150.56","10,226.69",1.4900%,"7,491",0,1,LinAct360
3,2023-03-21,2023-09-21,2023-09-21,"1,000,000","1,000,000",True,"1,007,513",CLP,"10,226.69","10,303.39",1.4700%,"7,513",0,1,LinAct360


## Colateral

Aprovechar de revisar los videos del notebook 03.

### Definiciones

¿Qué significa usar colateral en el contexto de las operaciones de derivados OTC? Defina los siguientes términos:

- **Margen inicial:** 
- **Margen de variación:** 
- **Close-out netting:**
- **Threshold:**
- **Colateral Elegible:**
- **Minimum transfer amount (MTA):**
- **Remuneración del colateral:**
- **Periodicidad de Cálculo:**

## Modelos de Tasa Corta

### Definiciones

#### Tasa Forward Lineal

Defina la tasa forward lineal y entregue una interpretación en términos de inversiones a los dos plazos que aparecen en la definición.

#### Tasa Forward Instantánea

Defina la tasa forward instantánea y todos los objetos que se requieren para llegar a estar definición.

### Ecuaciones

Para las próximas preguntas, suponga que, en la medida histórica:

$$
dr_t=m\left(r_t,t\right)dt+\sigma\left(r_t,t\right)dX_t
$$

#### $dZ_t$
Sea:

$$Z=Z\left(r_t, t, T\right)$$

un instrumento con un único payoff en $T\gt t$. Escriba la ecuación diferencial estocástica de $dZ_t$.

#### Ecuación Fundamental

Recuerde que todo instrumento de tasa de interés satisface:

$$
\frac{\partial Z}{\partial t}+\frac{\partial Z}{\partial r}m^*\left(r,t\right)+\frac{1}{2}\frac{\partial^2 Z}{\partial r^2}\sigma^2\left(r,t\right)-r Z=0
$$

¿Qué es $m^*$ y qué forma tiene? Describa que significan los términos que componen $m^*$.

## Construcción de Curva

Considere las siguientes cotizaciones de OIS.

In [8]:
quotes = pd.read_excel('guia.xlsx', sheet_name='quotes patas fijas')

In [9]:
quotes.style.format({'tasa fija': '{:.2%}'})

,plazo,tasa fija,convención tasa,periodicidad
0,0.500000,1.00%,Lin 30360,Cero cupón
1,1.000000,1.20%,Lin 30360,Cero cupón
2,2.000000,1.50%,Lin 30360,Semestral


Usando interpolación lineal calcule por bootstrapping las tasas a .5, 1.0 y 2.0 años.

## Valorización y Sensibilidad

Considere un OIS a 2Y recién contratado con tasa fija igual a 2%, un spread de 1% en la pata flotante y periodicidad semestral.

- Usando la librería, valorice el swap.
- Sin usar la librería, replique la valorización de la pata fija.
- Sin usar la librería, calcule la sensibilidad a un punto básico de la pata fija del swap a las 4 tasas cero cupón.